# Wavefront and WavefrontK

The `Wavefront` object represents real-space complex wavefront data of a photon field propoagating in the `z` direction with given carrier wavelength. 

The `WavefrontK` object is essentially the same fundammental data but in k-space. Some operations are more convenient in this space.

This shows some of the basic usages of these objects.

In [ ]:
from pmd_beamphysics.wavefront.wavefront import Wavefront
from pmd_beamphysics.wavefront.gaussian import add_gaussian

from scipy.constants import c
import numpy as np

import os

import matplotlib.pyplot as plt

## Basic usage

A `Wavefront` must be initialized with a 3D field array. Here for demonstration we will make 2x2 hot pixels inthe array.

Note the defaults for the other parameters are set to 1 m, and the photon energy is calculated.

In [ ]:
W = Wavefront(Ex=np.zeros((11, 11, 2)))
W.Ex[6:8, 7:9, :] = 1

W

Basic fluence and power plots are included:

In [ ]:
W.plot_fluence()

In [ ]:
W.plot_power()

Mean positions in meters

In [ ]:
W.mean_x

In [ ]:
W.mean_y

RMS sizes in meters. Note that because this is a single pixel, the pixel width is all that contributes.

In [ ]:
W.sigma_x

In [ ]:
W.sigma_y

The time-averaged field energy in J:

In [ ]:
W.energy

# Gaussian pulse 

Gaussian pulses can be implemented with the `add_gaussian` function. You are required to first populate the array before adding the Gaussian pulse

In [ ]:
W0 = Wavefront(
    Ex=np.zeros((101, 101, 51)),
    dx=10e-6,
    dy=10e-6,
    dz=10e-6,
    wavelength=1e-9,
)
w0 = 100e-6
zR = np.pi * w0**2 / W0.wavelength


W = W0.copy()
add_gaussian(W, z=0, w0=w0, energy=1.2345, sigma_z=50e-6)

W.plot_fluence()

In [ ]:
W.plot_power()

Check that the energy is what we requested:

In [ ]:
W.energy

Check the sums of the fluence profiles agree with this:

In [ ]:
Fx = W.fluence_profile_x
np.sum(Fx) * W.dx

In [ ]:
Fy = W.fluence_profile_y
np.sum(Fy) * W.dy

Check that the summed power agrees with the energy

In [ ]:
P = W.power
np.sum(P) * W.dz / c

Check the waist size is 100 µm that we originally requested:

In [ ]:
W.sigma_x * 2

## Drift propagation

The `drift_wavefront` function will propagate a `Wavefront` using Frenel propagation with FFT convolutions. Here we will propagate to 100 m. 

In [ ]:
W100 = W.drift(100)
W100.plot_fluence()

In [ ]:
%%time
Zlist = np.linspace(0, 100, 20)
Wlist = [W.drift(z) for z in Zlist]

sizes = np.array([w.sigma_x for w in Wlist])
sizes

In [ ]:
sigma_x0 = W.sigma_x

expected_w = sigma_x0 * np.sqrt(1 + (Zlist / zR) ** 2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Zlist, 1e6 * expected_w, label="expected")
ax.plot(Zlist, 1e6 * sizes, "--", label="propagated")

ax.set_xlabel(r"$z$ (m)")

ax.set_ylabel(r"$\sigma_x$ (µm)")
plt.legend()

## Drift with curvature correction

The default `drift_wavefont` does not resize the grid. Alternatively, you can specify a `curvature` in 1/m in the propagation that will resize the grid to help keep the spot within the domain. 

See the `advanced_drift` notebook for more details.


In [ ]:
W100_advancd = W.drift(100, curvature=1 / 50)
W100_advancd.plot_fluence()

# $M^2$ fit

The beam size squared equation is given by:

$$
w^2(z) = w_0^2 \left[ 1 + \left(\frac{M^2 \lambda}{\pi w_0^2} (z - z_0) \right)^2 \right]
$$

Where:
- $w(z)$: Beam radius at position \(z\).
- $w_0$: Beam waist radius (minimum beam size).
- $z_0$: Position of the beam waist.
- $z$: Position along the propagation axis.
- $M^2$: Beam quality factor.
- $\lambda$: Wavelength of the light.

In this equation:
- At $z = z_0$, the beam size is at its minimum: $w^2(z_0) = w_0^2$.
- As $z$ increases or decreases from $z_0$, the beam size squared grows quadratically, scaled by the $M^2$ parameter.


In [ ]:
z = Zlist
w2 = (2 * expected_w) ** 2
wavelength = W.wavelength

In [ ]:
from scipy.optimize import curve_fit


# Define the beam size squared function
def beam_size_squared(z, w0, z0, M2, wavelength=wavelength):
    k = M2 * wavelength / (np.pi * w0**2)  # Divergence coefficient
    return w0**2 * (1 + (k * (z - z0)) ** 2)


# Initial guesses for w0, z0, and M2
initial_guess = [1e-3, 0.0, 1.0]

# Curve fitting
popt, pcov = curve_fit(beam_size_squared, z, w2, p0=initial_guess)

# Extract fitted parameters
w0_fit, z0_fit, M2_fit = popt

# Print results
print(f"Fitted w0 (beam waist):     {w0_fit:10.9f} m")
print(f"Fitted z0 (waist position): {z0_fit:10.9f} m")
print(f"Fitted M^2 (beam quality):  {M2_fit:10.9f}")

# Plot the results
z_fit = np.linspace(min(z), max(z), 500)
w2_fit = beam_size_squared(z_fit, *popt)

plt.figure(figsize=(8, 6))
plt.scatter(z, w2, label="Data", color="blue", marker="o")
plt.plot(z_fit, w2_fit, label="Fit", color="red")
plt.xlabel("z (m)")
plt.ylabel("Beam Size Squared (w²) (m²)")
plt.legend()
plt.title("Beam Size Squared vs. z")
plt.grid()
plt.show()

# K-space 

We transform to k-space according to the Fourier transform convention:

Ẽ(kx,ky,kz) = 1/(2π)^(2/3) ∫∫∫ E(x,y,z) exp(-i kx x) exp(-i ky y) exp(-i kz z) dx dy dz

with units in V * m^2


In [ ]:
Wk = W.to_kspace()

Wk.plot_spectral_intensity()

The photon spectrum can be plotted:

In [ ]:
Wk.plot_photon_energy_spectrum()

This corresponds with the data:

In [ ]:
Wk.photon_energy_vec, Wk.photon_energy_spectrum

The energy should be the same as in r-space:

In [ ]:
Wk.energy

These are the rms angular sizes in radians:

In [ ]:
Wk.sigma_thetax

In [ ]:
Wk.sigma_thetay

Check that the angular sizes are preseved with the previous propagation

In [ ]:
[float(w.to_kspace().sigma_thetax) for w in Wlist]

In [ ]:
1 / W.sigma_x / W.k0 / 2

In [ ]:
Wk.sigma_kx

In [ ]:
1 / W.sigma_x / 2

# Conversion

## Genesis4

The `.write_genesis4` method will write to a Genesis4 native HDF5 file. Such files also be read in with `.from_genesis4`. 

In [ ]:
W.write_genesis4("genesis4_field.h5")

In [ ]:
W2 = Wavefront.from_genesis4("genesis4_field.h5")

Check that the results are close:

In [ ]:
np.allclose(W.Ex, W2.Ex)

In [ ]:
# Cleanup
os.remove("genesis4_field.h5")